# Introduction

It is becoming impossible to store data in a single computer, that's when parallel data analysis comes into play. Yoav Freund from UCSD. We will use Apache Spark with _pyspark_.

# Memory latency

We will learn how to use computers' clusters to efficiently process large amounts of data.

Before, let's understand the problem, what makes computation on very large datasets very slow?

At a high level, any computer has two main parts: CPU (central processing unit) and storage. If we need to multiply two numbers, initially the numbers are in storage; at the first step, the numbers are read from storage and into registers A and B in the CPU, then the numbers are multiplied and the result is stored in a third register C, finnaly C is written back to storage.

The time to complete one step is called step latency, and to complete all stel total latency:

1. Read A
2. Read B
3. C = A * B
4. Write C

With big data, most of the latency is memory latency (1,2,4), not computation (3)

Storage latency varies widely: main memory (RAM) is fast, while spinning memory is slow, memory that resides in a different computer depends on the load of the network

Big data analytics revolves around methods for organizing storage and computation in ways that maximize speed while minimizing cost

# Cache

Latency, size and price: we need to trade-off speed and storage size.

Caching is a way for combining fast and slow memory, to create storage that is both fast and large.

Consider a computer with a CPU, a fast and small cache (8 bytes) and a slow and large memory (80 bytes). The CPU first check the cache for the content, if it is, it can be retrieved quickly (cache hit), if not, then retrieval will be slow (cache miss).

The cache is effective is the hit rate is high.

Temporal locality: multiple accesses to same address within a shift time period

Spatial locality: multiple accesses to close-together addresses in short time period (eg. difference between two sums, counting words by sorting). The cache benefit from locality because memory is partitioned into blocks/lines rather than single bytes, moving a block of memory takes much less time than moving each byte individually. Memory locations that are close to each other are likely to fall in the same block, resulting in more cache hits.

How sorting improves the locality of word count

# Memory access locality

Access locality refers to the ability of software to make good use of the cache

Memory is broken into pages. Software that uses the same of neighboring pages repeatedly has good access locality. 
Hardware is designed to speed up such software

* Temporal locality: accessing the same element again and again

Suppose the task is compute a function - $f_\theta(x)$ - over a long sequence $x_1,x_2,...,x_n$, $\theta$ is a parameters vector (eg. weights of a neural network), the parameters $\theta$ are needed for each computation, if $\theta$ fits in the cache, access is fast. If $\theta$ does not fit the cache, each $x_i$ causes at least two cache misses

* Spatial locality: you don't access the same element, but the next element, and the next element etc.

Suppose the task is compute the function $\sum^{n-1}_{i=1}(x_i-x_{i+1})^2$ on $x_1,x_2,...,x_n$. Contrast two ways of storing it: i) a linked list (poor locality); ii) indexed array (good locality)

# Row-wise vs column-wise scanning

The way you traverse a 2D array effects speed

numpy arrays are, by default, organized in a row-major order

In [1]:
import numpy as np

In [2]:
a = np.array([range(1, 31)]).reshape([3, 10])
a

array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       [11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25, 26, 27, 28, 29, 30]])

$a[i,j]$ and $a[i,j+1]$ are placed in consecutive places in memory

$a[i,j]$ and $a[i+1,j]$ are 10 memory locations apart

This implies that scanning the array row by row is more local than scanning column by column (locality implies speed)

In [3]:
%%time
# scan column by column
s=0
for i in range(a.shape[1]): s+=sum(a[:,i])

CPU times: user 183 µs, sys: 39 µs, total: 222 µs
Wall time: 120 µs


In [4]:
%%time
# scan row by row
s=0
for i in range(a.shape[0]): s+=sum(a[i,:])

CPU times: user 128 µs, sys: 29 µs, total: 157 µs
Wall time: 60.3 µs


* Conclusions:

Traversing a numpy array column by column takes more time than row by row

The effect increases proportionaly to the number of elements in the array (square of one dimension)

# Measuring latency

Goals: measure the effects of caching in the wild and understand how to study long tail distributions

**Latency** is the time difference between the time at which the CPU is issuing a read or write command and the time the command is complete

This time is very short if the element is already in the L1 Cache and is very long if the element is in external memory (disk or SSD)

We'll test arrays of sizes [zero, 1kb, 1mb, 1gb, 10gb], we'll perform 100,000 read/write ops to random locations in the array, we'll analyze the distribution of the latencies as a function of the size of the array

Looking just at the mean and standard deviation of the latencies, we could assume we have a normal distribution, but in fact latency have a much longer tail (higher probability of a high latency) than a Gaussian distribution, which we could observe from the CDF

* Characterize sequential access

Random access degrades rapidly with the size of the block. Sequential access is much faster. We already saw that writting 10GB to disk sequentially takes 8.9s (less than 1s for 1Gb). Writting a 1TB disk at this rate would take ~16min

**Bandwith**: total number of GB that we can write in so many seconds. We are measuring bandwidth rather than latency. We say that it takes 8.9s to write 10GB to SSD, we are not saying that it takes $8.9e^{-10}$ to write a single byte, because many write operations are occuring in parallel

Byte-rate for writing large blocks is about 100MB/s to RAM

Byte-rate for writing large SSD blocks is about 1GB/s

The cost-effective solution is often a cluster of many cheap computers, each with many cores and break up data so that each computer has a small fraction of data

# Memory hierarchy

Cache in combination with a local access pattern leads to speed up in computation

Real systems have several levels of storage types: small and fast storage close to CPU are top hierarchy whereas large and slow storage further from CPU are botttom hierarchy, from the CPU registers, L1, L2 and L3 cache, main memory or RAM, disk storage and local area network.

**Cluster** is a collection of many computers. A data processing cluster is simply many computers linked through an ethernet connection, storage is shared. **Shuffling** is the mechanism of transferring data between computers. In Spark, the transferring mechanism is called **resilient distribution data set or RDD** 

# History of large scale computing

Super computers: Cray, Deep Blue, Blue Gene. Specialized hardware, extremely expensive, created to solve specialized important problems

Data centers: physical side of what we call cloud computing. Collection of commodity computers (computers of out daily use), vast number of computers (100,000s), created to provide computation for large and small organizations, computation as a commodity

Google 2003: Larry Page and Sergey Brin develop a method for storing very large files on multiple commodity computers, each file is broken into fixed-sized chunks. Each chunk is stored on multiiple chunk servers, the locations of the chunks is managed by the master computer

Properties of GFS/HDFS: commodity hardware (low cost per byte of storage), locality (data stored close to CPU), redundancy (can recover from server failures), abstraction (looks to user like standard file system, chunk mechanism is hidden)

**HDFS (Hadoop file server)** is a storage abstraction, a way to store files on many machines

**Map reduce** is a computation abstraction that works well with HDFS

It allows the programmer to specify parallel computation without kowning how the hardware is organized

Spark was developed by Matei Zaharia in 2014, Hadoop uses shared file system (disk), Spark uses shared main memory (faster, lower latency)